In [23]:
import json, re, string, sklearn, pickle, json
from sklearn.model_selection import train_test_split
from sklearn import ensemble, feature_extraction, linear_model, pipeline, metrics, naive_bayes, neural_network, discriminant_analysis, svm
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from datetime import datetime
import tensorflow as tf
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import keras
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from keras.utils import to_categorical
from keras import Sequential
from sklearn import model_selection
from keras.preprocessing.text import Tokenizer
import keras.layers
import pandas

In [10]:
def clean_string(string, var = False):
    """
    Tokenization/string cleaning for all datasets 
    """
    
    """removes bullet points"""
    string = re.sub("\\u2022", "", string) 
    string = re.sub("\\u00b7", "", string)
    
    """removes numbers"""
    string = re.sub(r"[0-9]","", string)
    
    """removes punctuation and other symbols"""
    filters='!"\#$%&()*+-/:;,<=>?@[\\]^`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    filters2='.'
    translate_dict2 = dict((c, "_") for c in filters2)
    translate_map2 = str.maketrans(translate_dict2)
    string = string.translate(translate_map)
    string = string.translate(translate_map2)
    
    """removes unnecessary spaces"""
    string = re.sub('\s+', ' ', string).strip()
    return string.strip() if var else string.strip().lower()

def generate_ngrams(s, n):
    # Convert to lowercases
    s = s.lower()
    
    # Replace all none alphanumeric characters with spaces
    #s = re.sub(r'[^a-zA-Z0-9\s]', '', s)
    
    # Break sentence in the token, remove empty tokens
    tokens = [token for token in list(s) if token != ""]
    # Use the zip function to help us generate n-grams
    # Concatentate the tokens into ngrams and return
    ngrams = [tokens[i:i+n] for i in range(len(tokens)-n + 1)]
    return ["".join(ngram) for ngram in ngrams]

def generate_list(s,list_n):
    ret = []
    for elem in list_n:
        ret += generate_ngrams(s,elem)
    return ret

def flatten(x):
    t = []
    for elem in x:
        curr = " ".join(elem)
        t+= [curr]
    return t

In [11]:
training_X_data = open("train_X_languages_homework.json.txt")
X_train_content= training_X_data.readlines()
X = [json.loads(obj)['text'] for obj in X_train_content]

training_Y_data = open("train_y_languages_homework.json.txt")
Y_train_content= training_Y_data.readlines()
Y = [json.loads(obj)['classification'] for obj in Y_train_content]
lang_encode = dict(zip(set(Y),range(len(Y))))

trainDF = pandas.DataFrame()
trainDF['text'] = X
trainDF['label'] = Y
trainDF['label_encode'] = [lang_encode[i] for i in Y]
trainDF

,text,label,label_encode
0,Lacus class fames,lorem,15
1,στην Πλατεία του,el,35
2,מפני שטעו לחשוב,he,52
3,• Florida •,da,47
4,. Siehe auch : Liste der Landschaften in Nordr...,de,53
5,Сицзян ) расположен,ru,41
6,. Deshalb wird,de,53
7,Uí Fiachrach · Uí Briúin · Uí Néill · Síl,hu,40
8,v srednjeveški latinščini,sl,14
9,menjadi embrio .,id,28


In [112]:
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', ngram_range=(1,3))
tfidf_vect_ngram_chars.fit(trainDF['text'])


TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [106]:
kfold = model_selection.StratifiedKFold(n_splits=8) 
# enumerate splits
count = 1
for train, test in kfold.split(trainDF['text'][:120000],trainDF["label"][:120000]):
    text_train = trainDF['text'].iloc[train]
    test_valid = trainDF['text'].iloc[test]
    y_train = trainDF["label"].iloc[train]
    y_valid = trainDF["label"].iloc[test]
    if count == 1:
        train_seq_x1 = tfidf_vect_ngram_chars.transform(text_train) 
        valid_seq_x1 = tfidf_vect_ngram_chars.transform(test_valid) 
        train_seq_y1 = y_train
        valid_seq_y1 = y_valid
    if count == 2:
        train_seq_x2 = tfidf_vect_ngram_chars.transform(text_train)
        valid_seq_x2 = tfidf_vect_ngram_chars.transform(test_valid) 
        train_seq_y2 = y_train
        valid_seq_y2 = y_valid
    if count == 3:
        train_seq_x3 = tfidf_vect_ngram_chars.transform(text_train)
        valid_seq_x3 = tfidf_vect_ngram_chars.transform(test_valid) 
        train_seq_y3 = y_train
        valid_seq_y3 = y_valid
    if count == 4:
        train_seq_x4 = tfidf_vect_ngram_chars.transform(text_train)
        valid_seq_x4 = tfidf_vect_ngram_chars.transform(test_valid) 
        train_seq_y4 = y_train
        valid_seq_y4 = y_valid
    if count == 5:
        train_seq_x5 = tfidf_vect_ngram_chars.transform(text_train)
        valid_seq_x5 = tfidf_vect_ngram_chars.transform(test_valid) 
        train_seq_y5 = y_train
        valid_seq_y5 = y_valid
    if count == 6:
        train_seq_x6 = tfidf_vect_ngram_chars.transform(text_train)
        valid_seq_x6 = tfidf_vect_ngram_chars.transform(test_valid) 
        train_seq_y6 = y_train
        valid_seq_y6 = y_valid
    if count == 7:
        train_seq_x7 = tfidf_vect_ngram_chars.transform(text_train)
        valid_seq_x7 = tfidf_vect_ngram_chars.transform(test_valid) 
        train_seq_y7 = y_train
        valid_seq_y7 = y_valid
    if count == 8:
        train_seq_x8 = tfidf_vect_ngram_chars.transform(text_train)
        valid_seq_x8 = tfidf_vect_ngram_chars.transform(test_valid) 
        train_seq_y8 = y_train
        valid_seq_y8 = y_valid
    count += 1
    

In [15]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, label_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on test dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return predictions,metrics.accuracy_score(predictions, label_valid)

In [91]:
log_regression = linear_model.LogisticRegression()
y_predictions_lr, accuracy_lr = train_model(log_regression,train_seq_x1,train_seq_y1,valid_seq_x1,valid_seq_y1)
accuracy_lr

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7916527990414698

In [18]:
naive_bayes_classifer = sklearn.naive_bayes.MultinomialNB()
y_predictions_nb, accuracy_nb = train_model(naive_bayes_classifer,train_seq_x2,train_seq_y2,valid_seq_x2,valid_seq_y2)
accuracy_nb

0.7180819180819181

In [107]:
svm_c = svm.LinearSVC()
y_predictions_svm, accuracy_svm = train_model(svm_c,train_seq_x4,train_seq_y4,valid_seq_x4,valid_seq_y4)
accuracy_svm

0.8109044857695128

In [90]:
svm_c = svm.LinearSVC()
y_predictions_svm, accuracy_svm = train_model(svm_c,train_seq_x4,train_seq_y4,valid_seq_x4,valid_seq_y4)
accuracy_svm

0.8329000866493368

In [108]:
sgdc = linear_model.SGDClassifier(loss='hinge',penalty='l2')
y_predictions_sgdc, accuracy_sgdc = train_model(sgdc,train_seq_x5,train_seq_y5,valid_seq_x5,valid_seq_y5)
accuracy_sgdc

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.773421350936854

In [26]:
svm_c2 = svm.LinearSVC()
y_predictions_svm2, accuracy_svm2 = train_model(svm_c2,train_seq_x6,train_seq_y6,valid_seq_x6,valid_seq_y6)
accuracy_svm2

0.7924893276414088

In [ ]:
print("hi")

In [28]:
log_regression2 = linear_model.LogisticRegression()
y_predictions_lr2, accuracy_lr2 = train_model(log_regression2,train_seq_x7,train_seq_y7,valid_seq_x7,valid_seq_y7)
accuracy_lr2

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.773870687929539

In [29]:
sgdc2 = linear_model.SGDClassifier(loss='hinge',penalty='l2')
y_predictions_sgdc2, accuracy_sgdc2 = train_model(sgdc2,train_seq_x8,train_seq_y8,valid_seq_x8,valid_seq_y8)
accuracy_sgdc2

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.7525043408574863

In [102]:
test_x = trainDF['text'][120000:]
test_y = trainDF["label"][120000:]
test_x_vect = tfidf_vect_ngram_chars.transform(test_x)


In [32]:
y1_c = log_regression.predict(test_x_vect)
y2_c = naive_bayes_classifer.predict(test_x_vect)
y3_c = mlp.predict(test_x_vect)
y4_c = svm_c.predict(test_x_vect)
y5_c = sgdc.predict(test_x_vect)
y6_c = svm_c2.predict(test_x_vect)
y7_c = log_regression2.predict(test_x_vect)
y8_c = sgdc2.predict(test_x_vect)

array(['it', 'sl', 'fr', ..., 'zh', 'hy', 'de'], dtype=object)

In [37]:
y_total = []
from statistics import mode
for elem in range(len(y1_c)):
    try:
        y_total += [mode([y1_c[elem],y2_c[elem],y3_c[elem],y4_c[elem],y5_c[elem],y6_c[elem],y7_c[elem],y8_c[elem]])]
    except statistics.StatisticsError:
        y_total += [y8_c[elem]]

In [39]:
y_total_mod = np.array(y_total)

In [41]:
metrics.accuracy_score(y_total_mod,test_y)

0.7759228343535522

In [93]:
svm_c1 = svm.LinearSVC()
y_predictions_svm1, accuracy_svm1 = train_model(svm_c1,train_seq_x1,train_seq_y1,valid_seq_x1,valid_seq_y1)
accuracy_svm1

0.7365333333333334

In [94]:
svm_c2 = svm.LinearSVC()
y_predictions_svm2, accuracy_svm2 = train_model(svm_c2,train_seq_x2,train_seq_y2,valid_seq_x2,valid_seq_y2)
accuracy_svm2

0.7448

In [95]:
svm_c3 = svm.LinearSVC()
y_predictions_svm3, accuracy_svm3 = train_model(svm_c3,train_seq_x3,train_seq_y3,valid_seq_x3,valid_seq_y3)
accuracy_svm3

0.7338666666666667

In [96]:
svm_c4 = svm.LinearSVC()
y_predictions_svm4, accuracy_svm4 = train_model(svm_c4,train_seq_x4,train_seq_y4,valid_seq_x4,valid_seq_y4)
accuracy_svm4

0.7293333333333333

In [97]:
svm_c5 = svm.LinearSVC()
y_predictions_svm5, accuracy_svm5 = train_model(svm_c5,train_seq_x5,train_seq_y5,valid_seq_x5,valid_seq_y5)
accuracy_svm5

0.7450666666666667

In [98]:
svm_c6 = svm.LinearSVC()
y_predictions_svm6, accuracy_svm6 = train_model(svm_c6,train_seq_x6,train_seq_y6,valid_seq_x6,valid_seq_y6)
accuracy_svm6

0.7456

In [99]:
svm_c7 = svm.LinearSVC()
y_predictions_svm7, accuracy_svm7 = train_model(svm_c7,train_seq_x7,train_seq_y7,valid_seq_x7,valid_seq_y7)
accuracy_svm7

0.7533333333333333

In [100]:
svm_c8 = svm.LinearSVC()
y_predictions_svm8, accuracy_svm8 = train_model(svm_c8,train_seq_x8,train_seq_y8,valid_seq_x8,valid_seq_y8)
accuracy_svm8

0.7504

In [103]:
y1 = svm_c1.predict(test_x_vect)
y2 = svm_c2.predict(test_x_vect)
y3 = svm_c3.predict(test_x_vect)
y4 = svm_c4.predict(test_x_vect)
y5 = svm_c5.predict(test_x_vect)
y6 = svm_c6.predict(test_x_vect)
y7 = svm_c7.predict(test_x_vect)
y8 = svm_c8.predict(test_x_vect)

y_total = []
from statistics import mode
for elem in range(len(y1)):
    try:
        y_total += [mode([y1[elem],y2[elem],y3[elem],y4[elem],y5[elem],y6[elem],y7[elem],y8[elem]])]
    except statistics.StatisticsError:
        y_total += [y8[elem]]


In [104]:
y_total_mod = np.array(y_total)
metrics.accuracy_score(y_total_mod,test_y)

0.7761083286959748

In [92]:
kfold = model_selection.KFold(n_splits=8) 
# enumerate splits
#train, test in kfold.split(trainDF['text'][:120000],trainDF["label"][:120000])
count = 1
for elem in range(0,8) :
    val = 15000
    text_train,test_valid,y_train,y_valid = model_selection.train_test_split(trainDF["text"][elem*val:(elem+1)*val],trainDF["label"][elem*val:(elem+1)*val])
#     text_train = trainDF['text'].iloc[elem*val:]
#     test_valid = trainDF['text'].iloc[test]
#     y_train = trainDF["label"].iloc[train]
#     y_valid = trainDF["label"].iloc[test]
    if count == 1:
        train_seq_x1 = tfidf_vect_ngram_chars.transform(text_train) 
        valid_seq_x1 = tfidf_vect_ngram_chars.transform(test_valid) 
        train_seq_y1 = y_train
        valid_seq_y1 = y_valid
    if count == 2:
        train_seq_x2 = tfidf_vect_ngram_chars.transform(text_train)
        valid_seq_x2 = tfidf_vect_ngram_chars.transform(test_valid) 
        train_seq_y2 = y_train
        valid_seq_y2 = y_valid
    if count == 3:
        train_seq_x3 = tfidf_vect_ngram_chars.transform(text_train)
        valid_seq_x3 = tfidf_vect_ngram_chars.transform(test_valid) 
        train_seq_y3 = y_train
        valid_seq_y3 = y_valid
    if count == 4:
        train_seq_x4 = tfidf_vect_ngram_chars.transform(text_train)
        valid_seq_x4 = tfidf_vect_ngram_chars.transform(test_valid) 
        train_seq_y4 = y_train
        valid_seq_y4 = y_valid
    if count == 5:
        train_seq_x5 = tfidf_vect_ngram_chars.transform(text_train)
        valid_seq_x5 = tfidf_vect_ngram_chars.transform(test_valid) 
        train_seq_y5 = y_train
        valid_seq_y5 = y_valid
    if count == 6:
        train_seq_x6 = tfidf_vect_ngram_chars.transform(text_train)
        valid_seq_x6 = tfidf_vect_ngram_chars.transform(test_valid) 
        train_seq_y6 = y_train
        valid_seq_y6 = y_valid
    if count == 7:
        train_seq_x7 = tfidf_vect_ngram_chars.transform(text_train)
        valid_seq_x7 = tfidf_vect_ngram_chars.transform(test_valid) 
        train_seq_y7 = y_train
        valid_seq_y7 = y_valid
    if count == 8:
        train_seq_x8 = tfidf_vect_ngram_chars.transform(text_train)
        valid_seq_x8 = tfidf_vect_ngram_chars.transform(test_valid) 
        train_seq_y8 = y_train
        valid_seq_y8 = y_valid
    count += 1

0                                         Lacus class fames
1                                          στην Πλατεία του
2                                           מפני שטעו לחשוב
3                                               • Florida •
4         . Siehe auch : Liste der Landschaften in Nordr...
5                                       Сицзян ) расположен
6                                            . Deshalb wird
7                 Uí Fiachrach · Uí Briúin · Uí Néill · Síl
8                                 v srednjeveški latinščini
9                                          menjadi embrio .
10                                 prevalentemente su testo
11                                           Anversa 1920 ·
12         Albania  •  Andora  •  Armenia  •  Austria  •...
13                                               acima , os
14                                          Krst pri Savici
15        čeprav so mu domači in strici duhovniki odtego...
16        ; den lovgivende makt , som li